In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asus4\\OneDrive\\Documents\\DEEP LEARNING PROJECT\\DL-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus4\\OneDrive\\Documents\\DEEP LEARNING PROJECT\\DL-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.cnnClassifier.constants import * 
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [9]:
import os 
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion: 
    def __init__(self,config:DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info : \n{headers}")
        else : 
            logger.info(f"file already exists of size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"extracted file at : {unzip_path}")

In [12]:
try: 
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-23 15:32:00,187]:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-10-23 15:32:00,192]:INFO:common:yaml file: params.yaml loaded successfully]
[2025-10-23 15:32:00,195]:INFO:common:created directory at : artifacts]


[2025-10-23 15:32:00,198]:INFO:common:created directory at : artifacts/data_ingestion]
[2025-10-23 15:32:14,159]:INFO:1325858822:artifacts\data_ingestion\data.zip download! with following info : 
Connection: close
Content-Length: 11634366
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "b9f8317cf6015bc25941f45a30e57d53cb23fa75c02220081f6c91d26eccd489"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4E23:265E85:25CF1:3E7DB:68F9FD1C
Accept-Ranges: bytes
Date: Thu, 23 Oct 2025 10:02:05 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210041-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1761213724.404075,VS0,VE1013
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 62abdedb3cfc47204724772cf0327ec8a09f3fd9
Expires: Thu,